In [5]:
import numpy as np
import pandas as pd
from math import exp

def r(t):
    """Instantaneous rate function r(t)."""
    return 0.05 / (1 + 2 * np.exp(-(1 + t)**2))

def simpsons_rule(f, a, b, n):
    """Approximate the integral of f from a to b using Simpson's Rule with n intervals (n must be even)."""
    if n % 2 != 0:
        raise ValueError("n must be even")
    h = (b - a) / n
    x = np.linspace(a, b, n + 1)
    fx = f(x)
    return h / 3 * (fx[0] + 2 * sum(fx[2:n:2]) + 4 * sum(fx[1:n:2]) + fx[n])

def discount_factors_convergence(t_values, n_values):
    """Compute discount factors at given t_values for each n in n_values."""
    data = {t: [] for t in t_values}
    for n in n_values:
        for t in t_values:
            integral = simpsons_rule(r, 0, t, n)
            discount = exp(-integral)
            data[t].append(round(discount, 8))
    df_conv = pd.DataFrame(data, index=n_values)
    df_conv.index.name = "n"
    return df_conv

# Define time points and n values to test
t_values = [0.5, 1.0, 1.5]
n_values = [2**i for i in range(2, 8)]  # n = 4, 8, 16, ..., 128

# Generate table
df_convergence = discount_factors_convergence(t_values, n_values)


In [8]:
print(df_convergence.to_latex(float_format="%.6f"))

\begin{tabular}{lrrr}
\toprule
 & 0.500000 & 1.000000 & 1.500000 \\
n &  &  &  \\
\midrule
4 & 0.982569 & 0.960514 & 0.937138 \\
8 & 0.982569 & 0.960516 & 0.937148 \\
16 & 0.982569 & 0.960516 & 0.937148 \\
32 & 0.982569 & 0.960516 & 0.937148 \\
64 & 0.982569 & 0.960516 & 0.937148 \\
128 & 0.982569 & 0.960516 & 0.937148 \\
\bottomrule
\end{tabular}



In [4]:
import numpy as np
import pandas as pd

# Define the instantaneous rate function r(t)
def r(t):
    return 0.05 / (1 + 2 * np.exp(-(1 + t)**2))

# Simpson's Rule
def simpson(f, a, b, n):
    h = (b - a) / n
    x = np.linspace(a, b, n + 1)
    fx = f(x)
    return h / 3 * (fx[0] + 2 * np.sum(fx[2:n:2]) + 4 * np.sum(fx[1:n:2]) + fx[n])

# Compute discount factor with adaptive Simpson's rule
def compute_discount_factor(t, tol=1e-6):
    n = 2
    I_old = simpson(r, 0, t, n)
    logs = [(n, np.exp(-I_old))]

    while True:
        n *= 2
        I_new = simpson(r, 0, t, n)
        disc = np.exp(-I_new)
        logs.append((n, disc))
        if abs(I_new - I_old) < tol:
            break
        I_old = I_new

    return pd.DataFrame(logs, columns=["n", f"D({t})"])

# Times to evaluate (in years)
times = [0.5, 1.0, 1.5, 2.0]
df_list = [compute_discount_factor(t) for t in times]

# Merge into one DataFrame
merged = df_list[0]
for df in df_list[1:]:
    merged = pd.merge(merged, df, on="n", how="outer")

merged = merged.sort_values("n").reset_index(drop=True)

# Format output for 6 decimals (except 2y with 8)
for col in merged.columns[1:]:
    precision = 8 if col == "D(2.0)" else 6

merged


,n,D(0.5),D(1.0),D(1.5),D(2.0)
0,2,0.982568,0.960434,0.936715,0.913405
1,4,0.982569,0.960514,0.937138,0.913981
2,8,NaN,0.960516,0.937148,0.914040
3,16,NaN,0.960516,0.937148,0.914041
4,32,NaN,NaN,NaN,0.914041


In [6]:
print(merged.to_latex(float_format="%.8f", index=False))

\begin{tabular}{rrrrr}
\toprule
n & D(0.5) & D(1.0) & D(1.5) & D(2.0) \\
\midrule
2 & 0.98256846 & 0.96043413 & 0.93671482 & 0.91340548 \\
4 & 0.98256855 & 0.96051378 & 0.93713849 & 0.91398132 \\
8 & NaN & 0.96051612 & 0.93714794 & 0.91403997 \\
16 & NaN & 0.96051626 & 0.93714845 & 0.91404123 \\
32 & NaN & NaN & NaN & 0.91404131 \\
\bottomrule
\end{tabular}

